# Projeto 2 - Ciência dos Dados

Nome: Nevton de Castro

Nome: Omar Dibo

Nome: Laura Perim 

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [2]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [102]:
import tweepy
import math
import os.path
import pandas as pd
import json
import re
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***@Devsegundoseme1***

In [4]:
#Dados de autenticação do twitter: authpass

#Coloque aqui o identificador da conta no twitter: @Devsegundoseme1

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:   
    data = json.load(fp)
    
#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [308]:
#Produto escolhido:
produto = 'civic'

#Quantidade mínima de mensagens capturadas:
n = 750
#Quantidade mínima de mensagens para a base de treinamento:
t = 500

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [310]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    if msg.full_text.lower()[0] != 'r' and msg.full_text.lower()[1] != 't' and msg.full_text.lower()[2] != ' ':
        msgs.append(msg.full_text.lower())
        i += 1
        if i > n:
                break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [318]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs1[:n])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs1[n:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()



,Treinamento,valor
7,honda civic lxl 2011\n* completo\n* automático...,0
13,carro usado honda civic lxr 2.0 flex aut. - 20...,0
20,honda civic 2020. o melhor preço de sp! consul...,0
45,leilão eletrónico\nencerramento: 16/09 às 17h0...,0
53,vale? honda civic 2020 | novidades | lançament...,0
60,carro usado honda civic exl 2.0 flex aut. - 20...,0
76,"civic lxs, automático, ano 2014, 69.000km e co...",0
85,minha garagem daqui a 13 anos\n- honda civic s...,0
135,olha que raridade está te esperando!!!😍😱🔥\nhon...,0
137,condor e nestlé sorteiam honda civic 0km: http...,0


___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [330]:
civic = pd.read_excel('civic.xlsx')
spm1 = civic[civic.valor==0] #tweets de spam ou venda
irr1 = civic[civic.valor==1] #tweets irrelevantes
rel1 = civic[civic.valor==2] #tweets de elogios ou criticas
irr = irr1['Treinamento'].tolist()
rel = rel1['Treinamento'].tolist()
spm = spm1['Treinamento'].tolist()

In [356]:
spm1.valor.value_counts() #40
rel1.valor.value_counts() #172
irr1.valor.value_counts() #537

1    537
Name: valor, dtype: int64

In [409]:
rel

['mas o corolla 2020 vai deixar de ser do vovo,e ser meu sonho de consumo, adeus civic',
 'acabou pra concorrência... 16km/l na cidade... lembrando q cruze, jetta e civic estão todos na casa de 120k... checkmate dos japoneses https://t.co/u5tpiwa06x',
 'civic preto é lindo demais mano',
 '@sergioftavares sempre foi. acho que a mudança tinha que ser mais agressiva como a do civic foi. a transei desse ainda lembra a do anterior.',
 '@egaby11 eu tenho medo de começar amar e a pessoa fazer eu comprar um civic.',
 '@filipexciv mano, sempre fui fã de honda civic... e desde 2016 q eu tenho e nunca tive motivos pra trocar, daí vou só atualizando!',
 'trabalhar pra pegar minha xj6 e meu civic',
 '@pretodoborel1 patrão né primo, absolut e rolê de civic só',
 '@gabiedpereira eu e você chegando nos role de honda civic 🤪',
 'dentre os civic, a versão que menos perdeu valor foi a ex (4,4%). #odia \n\nhttps://t.co/o1yfot0ehj',
 'casa vez que passo naquela esquina e vejo aquele civic si me sobe uma vo

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [421]:

nop = []
#nao vamos remover "#" nem "|"
for i in rel:
    punctuation = '[!\-.:?;,_+]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', i.lower())
    nop.append(text_subbed)


clean = []
for i in nop:     
    trash = 'civic'
    text_subbed = re.sub(trash, ' ', i)
    clean.append(text_subbed)

relfinal = []
for i in clean:
    e = pd.Series(i.split())
    for i in e:
        relfinal.append(i)
df = pd.DataFrame(relfinal)
reldata = df[0].value_counts(True)
reldata.head(20)

o          0.035815
um         0.027767
de         0.023340
que        0.022938
e          0.022938
meu        0.018109
é          0.017706
honda      0.017706
do         0.013280
//t        0.011670
https      0.011670
eu         0.010463
a          0.009658
pra        0.008853
na         0.008853
carro      0.008853
mais       0.008451
não        0.008048
com        0.007646
corolla    0.007646
Name: 0, dtype: float64

In [423]:
nop2 = []
for i in irr:     
    punctuation = '[!\-.:?;,_\+]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', i.lower())
    nop2.append(text_subbed)


clean2 = []
for i in nop2:     
    trash = 'civic'
    text_subbed = re.sub(trash, ' ', i)
    clean2.append(text_subbed)


relfinal2 = []
for i in clean2:
    e = pd.Series(i.split())
    for i in e:
        relfinal2.append(i)
df2 = pd.DataFrame(relfinal2)
irrdata = df2[0].value_counts(True)
irrdata.head(20)

um       0.026024
de       0.024830
o        0.024114
e        0.021368
que      0.021249
a        0.018861
do       0.017071
honda    0.015877
https    0.015877
//t      0.015877
eu       0.011102
no       0.011102
meu      0.010027
com      0.009192
na       0.009192
é        0.008953
não      0.008595
mais     0.007282
carro    0.007162
pra      0.006446
Name: 0, dtype: float64

In [425]:

nop3 = []
#nao vamos remover "#" nem "|"
for i in spm:     
    punctuation = '[!\-.:?;,_\+]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', i.lower())
    nop3.append(text_subbed)

clean3 = []
for i in nop3:     
    trash = 'civic'
    text_subbed = re.sub(trash, ' ', i)
    clean3.append(text_subbed)

relfinal3 = []
for i in clean3:
    e = pd.Series(i.split())
    for i in e:
        relfinal3.append(i)
df3 = pd.DataFrame(relfinal3)
spmdata = df3[0].value_counts(True)
spmdata.head(20)

https    0.034921
//t      0.034921
honda    0.029365
de       0.023810
carro    0.015873
00       0.015079
900      0.015079
flex     0.015079
r$       0.013492
2        0.011905
em       0.011905
ref      0.011905
0        0.011905
usado    0.011905
e        0.011905
do       0.011111
aut      0.009524
o        0.009524
com      0.009524
a        0.008730
Name: 0, dtype: float64

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [429]:
lista =[]
t = 1
for i in clean:
    e = pd.Series(i.split())
    for w in e:
        r = reldata[w]
        t = r*t
    i = []
    i.append(t)
    lista.append(i)
    t = 1
    

d = 0
for i in clean2:
    e = pd.Series(i.split())
    for w in e:
        if w in df[0].tolist():
            r = reldata[w]
        else:
            r=0
        t = r*t
    
        lista.append(t)
    t = 1
    


[1.671564779835507e-12]

In [279]:
t = 1
for i in clean:
    e = pd.Series(i.split())
    for w in e:
        if w in df2[0].tolist():
            r = irrdata[w]
        else:
            r=0
        t = r*t
    print(t)
    t = 1
    
for i in clean2:
    e = pd.Series(i.split())
    for w in e:
        r = irrdata[w]
        t = r*t
    print(t)
    t = 1

0.0
0.0
4.0173221702844436e-16
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0
0.0
0.0
0.0
0.0
0.0
0.0
6.171875481610678e-14
0.0
0.0
0.0
1.1217851001224873e-20
0.0
0.0
0.0
0.0
6.332646169503568e-20
0.0
0.0
0.0
0.0
0.0
0.0
1.6272882206265238e-17
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
4.288038017156491e-32
0.0
0.0
0.0
0.0
0.0
3.8098329482110953e-10
0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.80311342195256e-10
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.1529714628716354e-13
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2.953877642541899e-23
5.995464487343727e-39
7.719797235204347e-20
4.619152737867704e-17
4.261458763874365e-42
7.511053503298176e-114
7.30509589831217e-24
9.108365607428782e-17
7.481072117467573e-20
4.375203115036759e-40
1.0613316521295263e-108
8.742657156825048e-22
3.54

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**